# Memoria
## Grupo 10
### Integrantes
- Xiao Peng Ye
- Zhengyu Ye
- Juan Diego Valencia Marin

## Índice
1. Introducción
2. Algoritmo implementado
3. Funcionamiento específico
4. Comparación con otro algoritmo
5. Conclusiones
6. Bibliografía

## Introducción
Esta práctica se basa en la implementación de un algoritmo evolutivo y de su comparación con otro de una biblioteca estudiada en esta asignatura, en concreto se realizará el algoritmo de [**Evolución Diferencial (DE)**](https://es.wikipedia.org/wiki/Evoluci%C3%B3n_diferencial) y la biblioteca [**SciPy**](https://docs.scipy.org/doc/scipy/reference/) donde encontraremos este mismo algoritmo de evolución diferencial.

El algoritmo Evolución Diferencial es un método de optimización que está dentro de la computación evolutiva (rama de la inteligencia artificial inspirada en los mecanismos de la evolución biológica).

La biblioteca SciPy, que viene de las siglas Scientific Python, es una biblioteca de cálculo científico que proporciona funciones de utilidad para optimización, estadísticas y procesamiento de señales.


## Algoritmo implementado
El algoritmo de **Evolución Diferencial (DE)** ha sido el realizado en esta práctica, este algoritmo tiene una configuración específica, la *mutación se utiliza la estrategia (de/rand/1)*, *selección de forma uniforme*, *cruzamiento binomial* y un *reemplazo elitista y posición a posición*.

La idea del DE es generar una población de nuevas soluciones a partir de perturbar soluciones pertenecientes a la población de ese momento.

El funcionamiento de este algoritmo es el siguiente:

1. Crear una población inicial.
2. Determinar un individuo de la población en función de la estrategia de selección.
3. Generar nuevos individuos según el operador de mutación.
4. Aplicar el cruzamiento/recombinación.
5. Reemplazar el individuo seleccionado.
6. Volver al paso 2 hasta obtener la mejor solución.

## Funcionamiento específico
- **Uniform Selection.**

    Se escogen tres vectores diferentes de forma aleatoria dentro de la población obtenida, con los que trabajaremos junto al **Target vector** del que se hablará más adelante. A estos tres vectores se les llama **Donor vectors**.
    
	
- **Mutation (de/rand/1).**
	
    Se utiliza la estrategia de/rand/1, donde se escogen tres vectores diferentes de forma aleatoria, . A continuación se realiza la siguiente operación:
    
    V<sub>i,g</sub> = x<sub>r<sub>0</sub>,g</sub> + F(x<sub>r<sub>1</sub>,g</sub> − x<sub>r<sub>2</sub>,g</sub>)
         Donde:
         - F es una constante entre 0 y 2, 1 en nuestro caso.
         - V<sub>i,g</sub> es el resultado de la mutación.


- **Binomial Crossover.**

  En el cruzamiento binomial, en donde aparece el **trial vector (u<sub>i,g</sub>)**, creado por los elementos del target vector (x<sub>i,g</sub>) y del donor vector (v<sub>i,g</sub>), además de una probabilidad **CR**, 0.1 en nuestro caso.
  
  u<sub>i,g</sub> =
  
  v<sub>i,g</sub> si rand <= CR
         o
  x<sub>i,g</sub> si rand > CR
      Donde:
      - *rand* es un número aleatorio comprendido entre 0 y 1.
      - *CR* es la probabilidad antes descrita.


- **Elitist Replacement.**

	Este reemplazo es el más habitual, en el que se compara el *target vector* con el *trial vector* y aquel con la función de menor valor es el admitido en la siguiente generación.
    

## Comparación

En este apartado se procederá a realizar la comparación del algoritmo **DE** implementado por nosotros, con el de la biblioteca. Se incluirán los resultados de ambas implementaciones respresentados mediante tablas y algunos comentarios explicativos.

En los siguientes fragmentos, se importan las librerías y las funciones a utilizar, junto con la creación de ambos algoritmos.
En particular, se utiliza la optimización de SciPy, que proporciona funciones para maximizar o minimizar funciones objetivas. Y algunas de las [funciones de optimización](https://en.wikipedia.org/wiki/Test_functions_for_optimization) como la *sphere, ackley, rosenbrock,...* entre otras, dentro de la librería [Benchmarks](https://deap.readthedocs.io/en/master/api/benchmarks.html#deap.benchmarks.sphere). 

In [53]:
from scipy.optimize import differential_evolution
from group10.EA import EA
from benchmarks.functions import sphere, ackley, rosenbrock, rastrigin, griewank, schwefel_2_21,schwefel_2_22, schwefel_1_2, extended_f_10, bohachevsky, schaffer
import pandas as pd
import numpy as np
import pyade.sade as sade

In [54]:
def DE_scipy(m_function,bounds,p_size,iteration):
    result = differential_evolution(m_function, bounds, popsize=p_size, polish=False, maxiter=iteration,strategy="rand1bin")
    return result

In [55]:
def DE_propio(m_function,bounds,p_size,iteration):
    miEA = EA(m_function, bounds, p_size)
    miEA.run(iteration)
    bestGenome = miEA.best()
    return bestGenome

In [56]:
def DE_SADE(m_function,bounds,probsize,p_size,iteration):
    params = sade.get_default_params(dim=probsize)
    params['bounds'         ] = np.array([[bounds[0], bounds[1]]] * probsize)
    params['max_evals'      ] = p_size*iteration
    params['population_size'] = p_size
    params['func'           ] = m_function
    results = []
    # for i in range(iteration):
    _, fitness = sade.apply(**params)
    results.append(fitness)

    return results

En esta parte, se procede a (poner algo) en ambas implementaciones con las funciones de la librería benchmarks. **OJOOOOO!!!!**

In [57]:
benchmark=[sphere, ackley, rosenbrock, rastrigin, griewank, schwefel_2_21,
             schwefel_2_22, schwefel_1_2, extended_f_10, bohachevsky, schaffer]

In [58]:
resultMI={}

for func in benchmark:
    resultMI[func.__name__] = DE_propio(func, [(-100, 100)] * 10,50,100).solution

In [59]:
resultDE={}
for func in benchmark:
    resultDE[func.__name__] = DE_scipy(func, [(-100, 100)] * 10,50,1).x



In [60]:
resultSADE={}
for func in benchmark:
    resultSADE[func.__name__] = DE_SADE(func, (-100, 100), 10,50,1)



Ahora se procede a obtener los resultados y almacenarlos en un par de tablas para falicitar su visualización y comprensión. Se obtendrán algunos valores estadísticos como la media, desviación, mínimo, máximo y mediana.

In [61]:
ind=['sphere', 'ackley', 'rosenbrock', 'rastrigin', 'griewank', 'schwefel_2_21',
             'schwefel_2_22', 'schwefel_1_2', 'extended_f_10', 'bohachevsky', 'schaffer']
col=['avg','std','min','max','med']

In [62]:
df_b=pd.DataFrame(index=ind,columns=col)

In [63]:
df_m=pd.DataFrame(index=ind,columns=col)

In [64]:
df_d=pd.DataFrame(index=ind,columns=col)

Una vez creadas las tablas, se procede a guardar los datos para poder visualizarlos.

In [65]:
algNames = ["MI", "DE","SADE"]

results_avg = {}
results_std = {}
results_min = {}
results_max = {}
results_median = {}


for n in algNames:
    results_avg[n] = []
    results_std[n] = []
    results_min[n] = []
    results_max[n] = []
    results_median[n] = []


for func in benchmark:
    f = func.__name__
    results_avg["MI"].append(np.mean(resultMI[f]))
    results_avg["DE"].append(np.mean(resultDE[f]))
    results_avg["SADE"].append(np.mean(resultSADE[f]))
    results_std["MI"].append(np.std(resultMI[f]))
    results_std["DE"].append(np.std(resultDE[f]))
    results_std["SADE"].append(np.std(resultSADE[f]))
    results_min["MI"].append(np.min(resultMI[f]))
    results_min["DE"].append(np.min(resultDE[f]))
    results_min["SADE"].append(np.min(resultSADE[f]))
    results_max["MI"].append(np.max(resultMI[f]))
    results_max["DE"].append(np.max(resultDE[f]))
    results_max["SADE"].append(np.max(resultSADE[f]))
    results_median["MI"].append(np.median(resultMI[f]))
    results_median["DE"].append(np.median(resultDE[f]))
    results_median["SADE"].append(np.median(resultSADE[f]))

In [66]:
df_b.loc[:,'avg']=results_avg["MI"]
df_m.loc[:,'avg']=results_avg["DE"]
df_d.loc[:,'avg']=results_avg["SADE"]
df_b.loc[:,'std']=results_std["MI"]
df_m.loc[:,'std']=results_std["DE"]
df_d.loc[:,'std']=results_std["SADE"]
df_b.loc[:,'max']=results_max["MI"]
df_m.loc[:,'max']=results_max["DE"]
df_d.loc[:,'max']=results_max["SADE"]
df_b.loc[:,'min']=results_min["MI"]
df_m.loc[:,'min']=results_min["DE"]
df_d.loc[:,'min']=results_min["SADE"]
df_b.loc[:,'med']=results_median["MI"]
df_m.loc[:,'med']=results_median["DE"]
df_d.loc[:,'med']=results_median["SADE"]

### Visualización de las tablas

- **Tabla de DE de la biblioteca scipy:**

In [67]:
df_b

,avg,std,min,max,med
sphere,-0.283663,2.433232,-3.302627,3.822544,-0.897659
ackley,23.999892,54.051819,-68.000000,100.000000,28.000000
rosenbrock,-12.078787,29.614070,-100.000000,3.934982,-3.328594
rastrigin,1.545868,2.821244,-3.540751,7.678521,1.121028
griewank,-1.344801,1.492276,-4.149187,0.748810,-1.098967
schwefel_2_21,3.378821,9.417666,-13.392439,14.746390,1.818928
schwefel_2_22,0.612901,3.585757,-4.910817,9.292363,0.213889
schwefel_1_2,-0.517860,21.872414,-26.795034,37.342893,-8.267620
extended_f_10,0.695182,4.806391,-11.616317,8.189149,1.997681
bohachevsky,-0.171184,3.705893,-9.844351,3.310118,1.532343


- **Tabla de DE de la biblioteca SADE:**

In [68]:
df_b

,avg,std,min,max,med
sphere,-0.283663,2.433232,-3.302627,3.822544,-0.897659
ackley,23.999892,54.051819,-68.000000,100.000000,28.000000
rosenbrock,-12.078787,29.614070,-100.000000,3.934982,-3.328594
rastrigin,1.545868,2.821244,-3.540751,7.678521,1.121028
griewank,-1.344801,1.492276,-4.149187,0.748810,-1.098967
schwefel_2_21,3.378821,9.417666,-13.392439,14.746390,1.818928
schwefel_2_22,0.612901,3.585757,-4.910817,9.292363,0.213889
schwefel_1_2,-0.517860,21.872414,-26.795034,37.342893,-8.267620
extended_f_10,0.695182,4.806391,-11.616317,8.189149,1.997681
bohachevsky,-0.171184,3.705893,-9.844351,3.310118,1.532343


- **Tabla de DE con nuestra implementación:**

In [69]:
df_m

,avg,std,min,max,med
sphere,1.668487,25.453226,-42.059266,44.023816,9.854872
ackley,-1.498059,46.340006,-86.789390,75.195269,-13.841070
rosenbrock,-6.614378,32.191904,-61.239254,52.802604,0.475426
rastrigin,-0.785315,26.802924,-47.334065,38.852907,8.422518
griewank,-6.206126,27.871874,-51.821161,31.534402,-10.269287
schwefel_2_21,-8.550278,20.813815,-48.864954,19.937451,-7.424768
schwefel_2_22,-9.481708,45.620855,-90.473735,82.494146,-1.704723
schwefel_1_2,0.221065,32.604286,-45.421517,60.952512,-2.178406
extended_f_10,14.608415,19.080113,-11.705082,51.313485,13.715991
bohachevsky,-9.121662,24.162465,-55.960043,28.069295,-11.772329


### Poner algo!
Y por último, utilizaremos la librería [scikit-posthocs](https://scikit-posthocs.readthedocs.io/en/latest/) para...**OJOOOOO!!!** algo de Paired comparison method

In [70]:
import scikit_posthocs as sp

data = pd.DataFrame({"algs": ["MI" ]*len(results_avg["MI"]) +
                             ["DE"]*len(results_avg["DE"]) +
                            ["SADE"]*len(results_avg["SADE"]),
                     "vals": results_avg["MI"] +
                             results_avg["DE"] +
                            results_avg["SADE"]})

sp.posthoc_wilcoxon(data, val_col='vals', group_col='algs', p_adjust = 'holm')




,MI,DE,SADE
MI,1.000000,0.577148,0.134766
DE,0.577148,1.000000,0.041016
SADE,0.134766,0.041016,1.000000


## Conclusiones
Este trabajo se ha enfocado en la comparación de los resultados del algoritmo de **Evolución Diferencial (DE)** implementado por nosotros con unas estrategias y  métodos específicos, con el mismo algoritmo (DE) proveniente de la librería SciPy. En los resultados, observamos... (poner algo)

En cuanto al algoritmo DE, vemos su utilidad y nos damos cuenta de que los resultados son bastante buenos, aunque no los utilicemos de una forma óptima. Además, como ya veníamos de realizar el algoritmo genético (GA) del anterior trabajo, vemos que hay bastantes algoritmos dentro de la computación evolutiva y nos percatamos de la gran utilidad de ellos.

(Pasando a un enfoque más personal sobre este trabajo) (cambiar) sobre este trabajo, encontramos interesante la posibilidad de comparar estos algoritmos, de este modo hemos podido adentrarnos e indagar más en profundidad sobre ellos, conociendo así las soluciones óptimas y las que no son tan óptimas. 

## Bibliografía
https://es.wikipedia.org/wiki/Evoluci%C3%B3n_diferencial

https://docs.scipy.org/doc/scipy/reference/

https://docs.scipy.org/doc/scipy/reference/optimize.html

https://en.wikipedia.org/wiki/Differential_evolution

https://www.sciencedirect.com/science/article/pii/S2215098618323401#e0025

https://www.sciencedirect.com/science/article/abs/pii/S1568494609000325

https://www.hindawi.com/journals/jcse/2013/462706/